In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

In [3]:
from torch.utils.data import DataLoader
from dataloaders.shapenet.shapenet_loader import ShapeNet

categories = ['car']
path = "../data/ShapeNet"

test_dataset = ShapeNet(path, "test", 2048, categories, load_renders=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4)

Loading (test) renders for car (02958343):   0%|          | 0/704 [00:00<?, ?it/s]

In [4]:
from utils.hyperparams import load_hyperparams

hparams_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/hparams.yaml'

hparams = load_hyperparams(hparams_path)

In [5]:
model_args = {
    'voxel_size' : hparams['voxel_size'],
    'nfs' : hparams['nfs'], 
    'attn_chans' : hparams['attn_chans'], 
    'attn_start' : hparams['attn_start'], 
    'cross_attn_chans' : hparams['cross_attn_chans'], 
    'cross_attn_start' : hparams['cross_attn_start'], 
    'cross_attn_cond_dim' : hparams['cross_attn_cond_dim'],
}

model = SPVUnet(**model_args)
model = GSPVD(model=model)

In [6]:
steps = 500
scheduler = 'ddpm'
ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/{steps}-steps.ckpt'
ckpt = torch.load(ckpt_path, weights_only=False)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [7]:
model = model.cuda().eval()

In [14]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
from my_schedulers.ddim_scheduler import DDIMSparseScheduler
from utils.visualization import display_pointclouds_grid

if scheduler == 'ddim':
    sched = DDIMSparseScheduler(
        beta_min=hparams['beta_min'], 
        beta_max=hparams['beta_max'], 
        steps=steps, 
        init_steps=hparams['n_steps'],
        mode=hparams['mode'],
    )
else:
    sched = DDPMSparseScheduler(
        beta_min=hparams['beta_min'], 
        beta_max=hparams['beta_max'], 
        steps=steps, 
        init_steps=hparams['n_steps'],
        mode=hparams['mode'],
    )

In [13]:
from tqdm.auto import tqdm

all_ref_pc = []
all_gen_pc = []

i = 0
for datapoint in tqdm(test_loader):
    ref_pc = datapoint['pc']
    features = datapoint['render-features'].cuda()

    B, N, C = ref_pc.shape
    gen_pc = sched.sample(model, B, N, reference=features)

    mean = test_loader.dataset.mean
    std = test_leader.dataset.std

    ref_pc = ref_pc *
    
    all_ref_pc.append(ref_pc)
    all_gen_pc.append(gen_pc)

    i += 1
    # if i == 2:
    #     break

all_ref_pc = torch.cat(all_ref_pc).cuda()
all_gen_pc = torch.cat(all_gen_pc).cuda()

  0%|          | 0/22 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

...

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

In [10]:
from metrics.evaluation_metrics import compute_all_metrics, jsd_between_point_cloud_sets as JSD
from pprint import pprint

In [15]:
results = compute_all_metrics(all_ref_pc, all_gen_pc, batch_size=32)
results = {k: (v.cpu().detach().item()
               if not isinstance(v, float) else v) for k, v in results.items()}

pprint(results)

  0%|          | 0/704 [00:00<?, ?it/s]

  0%|          | 0/704 [00:00<?, ?it/s]

  0%|          | 0/704 [00:00<?, ?it/s]

{'1-NN-CD-acc': 0.8934659361839294,
 '1-NN-CD-acc_f': 0.7926136255264282,
 '1-NN-CD-acc_t': 0.9943181872367859,
 '1-NN-EMD-acc': 0.9730113744735718,
 '1-NN-EMD-acc_f': 0.9474431872367859,
 '1-NN-EMD-acc_t': 0.9985795617103577,
 'lgan_cov-CD': 0.36789771914482117,
 'lgan_cov-EMD': 0.4389204680919647,
 'lgan_mmd-CD': 0.028930295258760452,
 'lgan_mmd-EMD': 0.26099610328674316,
 'lgan_mmd_smp-CD': 0.06372430175542831,
 'lgan_mmd_smp-EMD': 0.8759151101112366}


In [12]:
jsd = JSD(all_gen_pc.cpu().numpy(), all_ref_pc.cpu().numpy())
pprint('JSD: {}'.format(jsd))

'JSD: 0.035144025796413736'


In [24]:
from metrics.chamfer_dist import ChamferFunction

def cham3D(sample, ref):
    dist1, dist2 = ChamferFunction.apply(sample, ref)
    return dist1, dist2 

import torch

a = torch.rand((1, 1024, 3))
b = a.clone()
d1, d2 = cham3D(b, a)

print(d1.sum(), d2.sum())

tensor(0., device='cuda:0') tensor(0., device='cuda:0')
